In [1]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
import joblib
import numpy as np
from datetime import datetime, timedelta

file_path = ''
models_dir = 'models'

best_xgb_model = None
preprocessor = None
features_to_use = []
historical_weekly_avg = None
unique_stores_depts_list = []
df_stores_load = None
last_historical_date = None

try:
    best_xgb_model = joblib.load(os.path.join(models_dir, 'best_xgb_model.joblib'))
    preprocessor = joblib.load(os.path.join(models_dir, 'preprocessor.joblib'))

    df_stores_load = pd.read_csv(os.path.join(file_path, 'data/raw/stores data-set.csv'))
    df_features_load = pd.read_csv(os.path.join(file_path, 'data/raw/Features data set.csv'))
    df_sales_load = pd.read_csv(os.path.join(file_path, 'data/raw/sales data-set.csv'))

    df_sales_load['Date'] = pd.to_datetime(df_sales_load['Date'], format='%d/%m/%Y', errors='coerce')
    df_features_load['Date'] = pd.to_datetime(df_features_load['Date'], format='%d/%m/%Y', errors='coerce')
    df_sales_load.dropna(subset=['Date'], inplace=True)
    df_features_load.dropna(subset=['Date'], inplace=True)

    df_sales_stores_load = pd.merge(df_sales_load, df_stores_load, on='Store', how='left')
    df_final_for_context = pd.merge(df_sales_stores_load, df_features_load, on=['Store', 'Date'], how='left')

    markdown_cols = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
    for col in markdown_cols:
        if col in df_final_for_context.columns:
            df_final_for_context[col] = df_final_for_context[col].fillna(0)

    economic_cols = ['CPI', 'Unemployment']
    for col in economic_cols:
        if col in df_final_for_context.columns:
            df_final_for_context[col] = df_final_for_context[col].interpolate(method='linear', limit_direction='both')
            if df_final_for_context[col].isnull().any():
                df_final_for_context[col] = df_final_for_context[col].fillna(df_final_for_context[col].mean())

    df_final_for_context = df_final_for_context[df_final_for_context['Weekly_Sales'] > 0]
    df_final_for_context['Date'] = pd.to_datetime(df_final_for_context['Date'], errors='coerce')
    df_final_for_context.dropna(subset=['Date'], inplace=True)

    df_final_for_context['Year'] = df_final_for_context['Date'].dt.year
    df_final_for_context['Month'] = df_final_for_context['Date'].dt.month
    df_final_for_context['Week'] = df_final_for_context['Date'].dt.isocalendar().week.astype(int)
    df_final_for_context['Day'] = df_final_for_context['Date'].dt.day
    df_final_for_context['DayOfWeek'] = df_final_for_context['Date'].dt.dayofweek
    df_final_for_context['DayOfYear'] = df_final_for_context['Date'].dt.dayofyear

    if 'IsHoliday_x' in df_final_for_context.columns and 'IsHoliday_y' in df_final_for_context.columns:
        df_final_for_context.rename(columns={'IsHoliday_x': 'IsHoliday'}, inplace=True)
        df_final_for_context.drop(columns=['IsHoliday_y'], inplace=True)
    elif 'IsHoliday_x' in df_final_for_context.columns:
        df_final_for_context.rename(columns={'IsHoliday_x': 'IsHoliday'}, inplace=True)
    elif 'IsHoliday_y' in df_final_for_context.columns:
        df_final_for_context.rename(columns={'IsHoliday_y': 'IsHoliday'}, inplace=True)

    if 'IsHoliday' in df_final_for_context.columns:
        df_final_for_context['IsHoliday_Flag'] = df_final_for_context['IsHoliday'].astype(int)

    df_final_for_context['SuperBowl'] = ((df_final_for_context['Month'] == 2) & (df_final_for_context['Week'].isin([6, 7])) | \
                                         (df_final_for_context['Month'] == 9) & (df_final_for_context['Week'].isin([36])) | \
                                         (df_final_for_context['Month'] == 11) & (df_final_for_context['Week'].isin([47])) | \
                                         (df_final_for_context['Month'] == 12) & (df_final_for_context['Week'].isin([51, 52]))).astype(int)
    df_final_for_context['IsHoliday'] = df_final_for_context['IsHoliday_Flag'].astype(bool)

    df_final_for_context['LaborDay'] = ((df_final_for_context['Month'] == 9) & (df_final_for_context['Week'].isin([36])) & (df_final_for_context['IsHoliday'] == True)).astype(int)
    df_final_for_context['Thanksgiving'] = ((df_final_for_context['Month'] == 11) & (df_final_for_context['Week'].isin([47])) & (df_final_for_context['IsHoliday'] == True)).astype(int)
    df_final_for_context['Christmas'] = ((df_final_for_context['Month'] == 12) & (df_final_for_context['Week'].isin([51, 52])) & (df_final_for_context['IsHoliday'] == True)).astype(int)

    historical_weekly_avg = df_final_for_context.groupby('Week')[['Temperature', 'Fuel_Price', 'CPI', 'Unemployment']].mean().reset_index()
    unique_stores_depts_list = [tuple(x) for x in df_final_for_context[['Store', 'Dept']].drop_duplicates().values]
    last_historical_date = df_final_for_context['Date'].max()

    features_to_use = [
        'Store', 'Dept', 'Size', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment',
        'IsHoliday_Flag',
        'Year', 'Month', 'Week', 'Day', 'DayOfWeek', 'DayOfYear',
        'SuperBowl', 'LaborDay', 'Thanksgiving', 'Christmas',
        'TotalMarkDown', 'HasAnyMarkDown'
    ]
    markdown_cols_existing_in_df_final_context = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5'] # Usar essa lista para preencher 'Has_MarkDownX' e 'MarkDownX'
    for col in markdown_cols_existing_in_df_final_context:
        if f'Has_{col}' not in features_to_use:
             features_to_use.append(f'Has_{col}')
        if col not in features_to_use:
            features_to_use.append(col)
    
    if 'Type' in df_stores_load.columns and 'Type' not in features_to_use: # df_stores_load contém a coluna Type
        features_to_use.append('Type')


    print("Modelos e contexto de dados carregados/reconstruídos para geração de previsões.")

except Exception as e:
    print(f"Erro na preparação dos dados ou carregamento de modelos: {e}")
    exit()

Modelos e contexto de dados carregados/reconstruídos para geração de previsões.


In [2]:
def generate_future_predictions_df(num_weeks_to_predict=52):
    
    prediction_start_date = last_historical_date + timedelta(weeks=1)
    prediction_end_date = prediction_start_date + timedelta(weeks=num_weeks_to_predict -1)

    future_dates = pd.date_range(start=prediction_start_date, end=prediction_end_date, freq='W-FRI')

    future_df = pd.DataFrame()
    for store_id, dept_id in unique_stores_depts_list:
        temp_df = pd.DataFrame({'Date': future_dates,
                                'Store': store_id,
                                'Dept': dept_id})
        future_df = pd.concat([future_df, temp_df], ignore_index=True)

    future_df['Year'] = future_df['Date'].dt.year
    future_df['Month'] = future_df['Date'].dt.month
    future_df['Week'] = future_df['Date'].dt.isocalendar().week.astype(int)
    future_df['Day'] = future_df['Date'].dt.day
    future_df['DayOfWeek'] = future_df['Date'].dt.dayofweek
    future_df['DayOfYear'] = future_df['Date'].dt.dayofyear

    future_df['IsHoliday'] = False
    future_df['IsHoliday_Flag'] = ((future_df['Month'] == 2) & (future_df['Week'].isin([6, 7])) | \
                                    (future_df['Month'] == 9) & (future_df['Week'].isin([36])) | \
                                    (future_df['Month'] == 11) & (future_df['Week'].isin([47])) | \
                                    (future_df['Month'] == 12) & (future_df['Week'].isin([51, 52]))).astype(int)
    future_df['IsHoliday'] = future_df['IsHoliday_Flag'].astype(bool)

    future_df['SuperBowl'] = ((future_df['Month'] == 2) & (future_df['Week'].isin([6, 7])) & (future_df['IsHoliday'] == True)).astype(int)
    future_df['LaborDay'] = ((future_df['Month'] == 9) & (future_df['Week'].isin([36])) & (future_df['IsHoliday'] == True)).astype(int)
    future_df['Thanksgiving'] = ((future_df['Month'] == 11) & (future_df['Week'].isin([47])) & (future_df['IsHoliday'] == True)).astype(int)
    future_df['Christmas'] = ((future_df['Month'] == 12) & (future_df['Week'].isin([51, 52])) & (future_df['IsHoliday'] == True)).astype(int)

    future_df = pd.merge(future_df, historical_weekly_avg, on='Week', how='left')

    for col in markdown_cols:
        if col in features_to_use:
            future_df[col] = 0.0
        if f'Has_{col}' in features_to_use:
            future_df[f'Has_{col}'] = 0
    if 'TotalMarkDown' in features_to_use:
        future_df['TotalMarkDown'] = 0.0
    if 'HasAnyMarkDown' in features_to_use:
        future_df['HasAnyMarkDown'] = 0

    future_df = pd.merge(future_df, df_stores_load[['Store', 'Type', 'Size']].drop_duplicates(), on='Store', how='left')

    if 'Type' in future_df.columns:
        future_df['Type'] = future_df['Type'].astype('category')

    future_X_raw = future_df[features_to_use]
    X_future_processed = preprocessor.transform(future_X_raw)
    if hasattr(X_future_processed, 'toarray'):
        X_future_processed = X_future_processed.toarray()

    future_predictions = best_xgb_model.predict(X_future_processed)
    future_predictions[future_predictions < 0] = 0

    df_future_predictions = future_df[['Store', 'Dept', 'Date']].copy()
    df_future_predictions['Predicted_Weekly_Sales'] = future_predictions
    
    return df_future_predictions

In [3]:
print("\n=== Início do Módulo de Otimização de Estoque ===")

print("\nGerando previsões futuras para o ano completo...")
df_future_predictions = generate_future_predictions_df(num_weeks_to_predict=52)
print(f"DataFrame de previsões futuras gerado. Shape: {df_future_predictions.shape}")
print(df_future_predictions.head())


=== Início do Módulo de Otimização de Estoque ===

Gerando previsões futuras para o ano completo...
DataFrame de previsões futuras gerado. Shape: (172796, 4)
   Store  Dept       Date  Predicted_Weekly_Sales
0      1     1 2012-11-02            36898.437500
1      1     1 2012-11-09            16620.400391
2      1     1 2012-11-16            17820.150391
3      1     1 2012-11-23            20685.056641
4      1     1 2012-11-30            25611.126953


In [4]:

#Calculando a média de vendas previstas por Loja e Departamento
average_sales_per_sd = df_future_predictions.groupby(['Store', 'Dept'])['Predicted_Weekly_Sales'].mean().reset_index()
average_sales_per_sd.rename(columns={'Predicted_Weekly_Sales': 'Avg_Predicted_Weekly_Sales'}, inplace=True)

In [5]:
#Merge de volta com o df_future_predictions
df_future_predictions = pd.merge(df_future_predictions, average_sales_per_sd, on=['Store', 'Dept'], how='left')

print("\nMédias de vendas previstas por Loja/Departamento calculadas.")
print(df_future_predictions.head())


Médias de vendas previstas por Loja/Departamento calculadas.
   Store  Dept       Date  Predicted_Weekly_Sales  Avg_Predicted_Weekly_Sales
0      1     1 2012-11-02            36898.437500                23059.716797
1      1     1 2012-11-09            16620.400391                23059.716797
2      1     1 2012-11-16            17820.150391                23059.716797
3      1     1 2012-11-23            20685.056641                23059.716797
4      1     1 2012-11-30            25611.126953                23059.716797


In [6]:
#Aplicando Lógica de Recomendação de Estoque

THRESHOLD_INCREASE = 0.20 # 20% acima da média para recomendar aumento
THRESHOLD_DECREASE = 0.15 # 15% abaixo da média para recomendar redução

def get_stock_recommendation(row):
    predicted = row['Predicted_Weekly_Sales']
    avg = row['Avg_Predicted_Weekly_Sales']

    if predicted > avg * (1 + THRESHOLD_INCREASE):
        return 'Aumentar Estoque'
    elif predicted < avg * (1 - THRESHOLD_DECREASE):
        return 'Reduzir Estoque'
    else:
        return 'Manter Estoque'

In [7]:
df_future_predictions['Stock_Recommendation'] = df_future_predictions.apply(get_stock_recommendation, axis=1)

print("\nRecomendações de estoque geradas (amostra):")
print(df_future_predictions[['Store', 'Dept', 'Date', 'Predicted_Weekly_Sales', 'Avg_Predicted_Weekly_Sales', 'Stock_Recommendation']].head(10))

#Relatório/Insights (Resumo das Recomendações)
print("\n=== Resumo das Recomendações de Estoque ===")
print("Contagem de Recomendações:")
print(df_future_predictions['Stock_Recommendation'].value_counts())

print("\nTop 10 Semanas/Loja/Departamento para 'Aumentar Estoque' (maior desvio):")
top_increase_rec = df_future_predictions[df_future_predictions['Stock_Recommendation'] == 'Aumentar Estoque'].copy()
top_increase_rec['Deviation_from_Avg'] = (top_increase_rec['Predicted_Weekly_Sales'] / top_increase_rec['Avg_Predicted_Weekly_Sales']) - 1
print(top_increase_rec.sort_values(by='Deviation_from_Avg', ascending=False).head(10)[['Store', 'Dept', 'Date', 'Predicted_Weekly_Sales', 'Deviation_from_Avg']])

print("\nTop 10 Semanas/Loja/Departamento para 'Reduzir Estoque' (maior desvio):")
top_decrease_rec = df_future_predictions[df_future_predictions['Stock_Recommendation'] == 'Reduzir Estoque'].copy()
top_decrease_rec['Deviation_from_Avg'] = 1 - (top_decrease_rec['Predicted_Weekly_Sales'] / top_decrease_rec['Avg_Predicted_Weekly_Sales'])
print(top_decrease_rec.sort_values(by='Deviation_from_Avg', ascending=False).head(10)[['Store', 'Dept', 'Date', 'Predicted_Weekly_Sales', 'Deviation_from_Avg']])

print("\n=== Módulo de Otimização de Estoque Concluído ===")


Recomendações de estoque geradas (amostra):
   Store  Dept       Date  Predicted_Weekly_Sales  Avg_Predicted_Weekly_Sales  \
0      1     1 2012-11-02            36898.437500                23059.716797   
1      1     1 2012-11-09            16620.400391                23059.716797   
2      1     1 2012-11-16            17820.150391                23059.716797   
3      1     1 2012-11-23            20685.056641                23059.716797   
4      1     1 2012-11-30            25611.126953                23059.716797   
5      1     1 2012-12-07            23989.419922                23059.716797   
6      1     1 2012-12-14            37208.308594                23059.716797   
7      1     1 2012-12-21            41732.515625                23059.716797   
8      1     1 2012-12-28            30802.324219                23059.716797   
9      1     1 2013-01-04            16351.446289                23059.716797   

  Stock_Recommendation  
0     Aumentar Estoque  
1      Reduzi